<br>

<div align=center><font color=maroon size=6><b>Better performance with tf.function</b></font></div>

<br>

<font size=4><b>References:</b></font>
1. TF2 Core: <a href="https://www.tensorflow.org/guide" style="text-decoration:none;">TensorFlow Guide</a> 
    * `TensorFlow > Learn > TensorFlow Core > `Guide > <a href="https://www.tensorflow.org/guide/function" style="text-decoration:none;">Better performance with tf.function</a>
        * Run in <a href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/function.ipynb" style="text-decoration:none;">Google Colab</a>

<br>
<br>
<br>

In TensorFlow 2, [eager execution](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/eager.ipynb) is turned on by default. The user interface is intuitive and flexible (running one-off operations is much easier and faster), but this can come <font style="color:maroon;font-size:110%">at the expense of performance and deployability.</font>

<font style="color:maroon;font-size:110%">You can use `tf.function` to make graphs out of your programs. It is a transformation tool that creates Python-independent dataflow graphs out of your Python code. This will help you create **performant and portable models, and it is required to use `SavedModel`**.</font>

This guide will help you conceptualize how `tf.function` works under the hood, so you can use it effectively.

<font style="color:maroon;font-size:130%;line-height:25px">The main takeaways and recommendations are:

- Debug in eager mode, then decorate with `@tf.function`.
- Don't rely on **Python side effects** like **object mutation** or **list appends**.
- `tf.function` works best with **TensorFlow ops**; NumPy and Python calls are converted to **constants**.
</font>

In [1]:
import tensorflow as tf

In [2]:
print(tf.__version__)

2.8.0


<br>

Define a helper function to demonstrate the kinds of errors you might encounter:

In [3]:
import traceback
import contextlib

# Some helper code to demonstrate the kinds of errors you might encounter.
@contextlib.contextmanager
def assert_raises(error_class):
    try:
        yield
    except error_class as e:
        print('Caught expected exception \n  {}:'.format(error_class))
        traceback.print_exc(limit=2)
    except Exception as e:
        raise e
    else:
        raise Exception('Expected {} to be raised but no error was raised!'.format(error_class))

<br>
<br>
<br>

## Basics

### Usage

<font style="color:maroon;font-size:110%">A **`Function`** you define (for example by applying the `@tf.function` decorator) is **just like a core TensorFlow operation**: You can execute it eagerly; you can compute gradients; and so on.</font>

<br>

In [4]:
@tf.function  # The decorator converts `add` into a `Function`.
def add(a, b):
    return a + b

add(tf.ones([2, 2]), tf.ones([2, 2]))  #  [[2., 2.], [2., 2.]]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 2.],
       [2., 2.]], dtype=float32)>

In [5]:
v = tf.Variable(1.0)
with tf.GradientTape() as tape:
    result = add(v, 1.0)

tape.gradient(result, v)

<tf.Tensor: shape=(), dtype=float32, numpy=1.0>

<br>

<font style="color:maroon;font-size:110%">You can use `Function`s inside other `Function`s.</font>

In [6]:
@tf.function
def dense_layer(x, w, b):
    return add(tf.matmul(x, w), b)

dense_layer(tf.ones([3, 2]), tf.ones([2, 2]), tf.ones([2]))

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[3., 3.],
       [3., 3.],
       [3., 3.]], dtype=float32)>

<br>

<font style="color:maroon;font-size:110%">`Function`s can be faster than eager code, especially for graphs with many small ops. But for graphs with a few expensive ops (like convolutions), you may not see much speedup.</font>


In [7]:
import timeit
conv_layer = tf.keras.layers.Conv2D(100, 3)

@tf.function
def conv_fn(image):
    return conv_layer(image)

image = tf.zeros([1, 200, 200, 100])

# Warm up
conv_layer(image)   # Eager mode
conv_fn(image)      # Graph mode (/Function)
print("Eager conv:", timeit.timeit(lambda: conv_layer(image), number=10))

print("Function conv:", timeit.timeit(lambda: conv_fn(image), number=10))

print("Note how there's not much difference in performance for convolutions")


Eager conv: 0.0030562000000013967
Function conv: 0.003987099999999799
Note how there's not much difference in performance for convolutions


<br>
<br>

### Tracing

This section exposes how `Function` works under the hood, including implementation details *which may change in the future*. However, <font style="color:maroon;font-size:110%">once you understand why and when tracing happens, it's much easier to use `tf.function` effectively!</font>

<br>

#### What is "tracing"?

<font style="color:maroon;font-size:120%">A `Function` runs your program in a [TensorFlow Graph](https://www.tensorflow.org/guide/intro_to_graphs#what_are_graphs). However, a `tf.Graph` cannot represent all the things that you'd write in an eager TensorFlow program. For instance,</font> 
* **Python supports polymorphism, but `tf.Graph` requires its inputs to have a specified data type and dimension**. 
* Or you may perform side tasks like reading command-line arguments, raising an error, or working with a more complex Python object; none of these things can run in a `tf.Graph`.

<font style="color:maroon;font-size:110%">`Function` bridges this gap by separating your code in two stages:

  **1)**  In the first stage, referred to as "**tracing**", `Function` creates a new `tf.Graph`. <ins>Python code runs normally</ins>, but all TensorFlow operations (like adding two Tensors) are ***deferred***: they are captured by the `tf.Graph` and not run.

  **2)** In the second stage, a `tf.Graph` which contains everything that was deferred in the first stage is run. This stage is much faster than the tracing stage.

Depending on its inputs, `Function` will not always run the first stage when it is called.  See ["Rules of tracing"](#rules_of_tracing) below to get a better sense of how it makes that determination. **Skipping the first stage and only executing the second stage is what gives you TensorFlow's high performance**.</font>

When `Function` does decide to trace, the tracing stage is immediately followed by the second stage, so calling the `Function` both creates and runs the `tf.Graph`. Later you will see how you can run only the tracing stage with [`get_concrete_function`](#obtaining_concrete_functions).

In [8]:
@tf.function
def double(a):
    print("Tracing with", a)
    return a + a

print(double(tf.constant(1)))
print()

print(double(tf.constant(1.1)))
print()

print(double(tf.constant("a")))
print()

Tracing with Tensor("a:0", shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)

Tracing with Tensor("a:0", shape=(), dtype=float32)
tf.Tensor(2.2, shape=(), dtype=float32)

Tracing with Tensor("a:0", shape=(), dtype=string)
tf.Tensor(b'aa', shape=(), dtype=string)



<br>

<font style="color:maroon;font-size:110%">**Note that**</font> if you repeatedly call a `Function` with the same argument type, TensorFlow will skip the tracing stage and reuse a previously traced graph, as the generated graph would be identical.

In [9]:
# This doesn't print 'Tracing with ...'
print(double(tf.constant("b")))

tf.Tensor(b'bb', shape=(), dtype=string)


<br>

<font style="color:maroon;font-size:110%">You can use `pretty_printed_concrete_signatures()` to see all of the available traces:</font>

In [10]:
print(double.pretty_printed_concrete_signatures())

double(a)
  Args:
    a: int32 Tensor, shape=()
  Returns:
    int32 Tensor, shape=()

double(a)
  Args:
    a: float32 Tensor, shape=()
  Returns:
    float32 Tensor, shape=()

double(a)
  Args:
    a: string Tensor, shape=()
  Returns:
    string Tensor, shape=()


<br>

##### <font style="color:maroon;font-size:110%">Terminology</font>

So far, you've seen that `tf.function` creates a cached, dynamic dispatch layer over TensorFlow's graph tracing logic. To be more specific about the terminology:
<font style="color:maroon;font-size:115%">

- A `tf.Graph` is the raw, language-agnostic `(语言无关的)`, portable representation of a TensorFlow computation.
- A `ConcreteFunction` wraps a `tf.Graph`.
- A `Function` manages a cache of `ConcreteFunction`s and picks the right one for your inputs.
- `tf.function` wraps a Python function, returning a `Function` object.
- **Tracing** creates a `tf.Graph` and wraps it in a `ConcreteFunction`, also known as a **trace.**
</font>

<br>

#### <font style="color:maroon;font-size:120%">Rules of tracing</font>

When called, a `Function` matches the call arguments to existing `ConcreteFunction`s using `tf.types.experimental.TraceType` of each argument. <font style="color:maroon;font-size:110%">If a matching `ConcreteFunction` is found, the call is dispatched to it. If no match is found, a new `ConcreteFunction` is traced.

If multiple matches are found, the most specific signature is chosen. Matching is done by [subtyping](https://en.wikipedia.org/wiki/Subtyping)</font>, much like normal function calls in C++ or Java, for instance. For example, `TensorShape([1, 2])` is a subtype of `TensorShape([None, None])` and so a call to the tf.function with `TensorShape([1, 2])` can be dispatched to the `ConcreteFunction` produced with `TensorShape([None, None])` but if a `ConcreteFunction` with `TensorShape([1, None])` also exists then it will prioritized since it is more specific.

<br>

<font style="color:maroon;font-size:125%">The `TraceType` is determined from input arguments as follows:</font>
* For `Tensor`, the type is parameterized by the `Tensor`'s `dtype` and `shape`; ranked shapes are a subtype of unranked shapes; fixed dimensions are a subtype of unknown dimensions
* For `Variable`, the type is similar to `Tensor`, but also includes a unique resource ID of the variable, necessary to correctly wire control dependencies
* For Python primitive values, the type corresponds to the **value** itself. For example, the `TraceType` of the value `3` is `LiteralTraceType<3>`, not `int`.
* For Python ordered containers such as `list` and `tuple`, etc., the type is parameterized by the types of their elements; for example, the type of `[1, 2]` is `ListTraceType<LiteralTraceType<1>, LiteralTraceType<2>>` and the type for   `[2, 1]` is `ListTraceType<LiteralTraceType<2>, LiteralTraceType<1>>` which is different.
* For Python mappings such as `dict`, the type is also a mapping from the same keys but to the types of values instead the actual values. For example, the type of `{1: 2, 3: 4}`, is `MappingTraceType<<KeyValue<1, LiteralTraceType<2>>>, <KeyValue<3, LiteralTraceType<4>>>>`. However, unlike ordered containers, `{1: 2, 3: 4}` and `{3: 4, 1: 2}` have equivalent types.
* For Python objects which implement the `__tf_tracing_type__` method, the type is whatever that method returns
* For any other Python objects, the type is a generic `TraceType` which uses the object's Python equality and hashing for matching. (Note: It relies on [weakref](https://docs.python.org/3/library/weakref.html) to the object and hence only works as long as the object is in scope/not deleted.)


<font style="color:maroon;font-size:110%">**Note**: `TraceType` is based on the `Function` input parameters so changes to global and [free variables](https://docs.python.org/3/reference/executionmodel.html#binding-of-names) alone will not create a new trace. See [this section](#depending_on_python_global_and_free_variables) for recommended practices when dealing with Python global and free variables.</font>

<br>
<br>

### Controlling retracing

Retracing, which is when your `Function` creates more than one trace, helps ensures that TensorFlow generates correct graphs for each set of inputs. <font style="color:maroon;font-size:110%">However, **tracing is an expensive operation!** If your `Function` retraces a new graph for every call, you'll find that your code executes more slowly than if you didn't use `tf.function`.</font>

<font style="color:maroon;font-size:110%">To control the tracing behavior, you can use the following techniques:</font>

#### Pass a fixed <font color=maroon>**input_signature**</font> to <font color=maroon>**tf.function**</font>

In [11]:
@tf.function(input_signature=(tf.TensorSpec(shape=[None], dtype=tf.int32),))
def next_collatz(x):
    print("Tracing with", x)
    return tf.where(x % 2 == 0,  x // 2,  3 * x + 1)

print(next_collatz(tf.constant([1, 2])))

# You specified a 1-D tensor in the input signature, so this should fail.
with assert_raises(ValueError):
    next_collatz(tf.constant([[1, 2], [3, 4]]))   # 2-D

# You specified an int32 dtype in the input signature, so this should fail.
with assert_raises(ValueError):
    next_collatz(tf.constant([1.0, 2.0]))         # float32


Tracing with Tensor("x:0", shape=(None,), dtype=int32)
tf.Tensor([4 1], shape=(2,), dtype=int32)
Caught expected exception 
  <class 'ValueError'>:
Caught expected exception 
  <class 'ValueError'>:


Traceback (most recent call last):
  File "C:\Users\18617\AppData\Local\Temp/ipykernel_12952/1818809857.py", line 8, in assert_raises
    yield
  File "C:\Users\18617\AppData\Local\Temp/ipykernel_12952/1833497163.py", line 10, in <module>
    next_collatz(tf.constant([[1, 2], [3, 4]]))   # 2-D
ValueError: Python inputs incompatible with input_signature:
  inputs: (
    tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32))
  input_signature: (
    TensorSpec(shape=(None,), dtype=tf.int32, name=None)).
Traceback (most recent call last):
  File "C:\Users\18617\AppData\Local\Temp/ipykernel_12952/1818809857.py", line 8, in assert_raises
    yield
  File "C:\Users\18617\AppData\Local\Temp/ipykernel_12952/1833497163.py", line 14, in <module>
    next_collatz(tf.constant([1.0, 2.0]))         # float32
ValueError: Python inputs incompatible with input_signature:
  inputs: (
    tf.Tensor([1. 2.], shape=(2,), dtype=float32))
  input_signature: (
    TensorSpec(shape=(None,), dtype=tf.int32, nam

<br>
<br>

#### Use unknown dimensions for flexibility

<font style="color:maroon;font-size:110%">Since TensorFlow matches tensors based on their shape, using a `None` dimension as a wildcard will allow `Function`s to reuse traces for variably-sized input.</font> 

**Variably-sized input** can occur if you have sequences of different length, or images of different sizes for each batch (See the [Transformer](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/text/transformer.ipynb) and [Deep Dream](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/generative/deepdream.ipynb) tutorials for example).

In [12]:
@tf.function(input_signature=(tf.TensorSpec(shape=[None], dtype=tf.int32),))
def g(x):
    print('Tracing with', x)
    return x

# No retrace!
print(g(tf.constant([1, 2, 3])))
print(g(tf.constant([1, 2, 3, 4, 5])))

Tracing with Tensor("x:0", shape=(None,), dtype=int32)
tf.Tensor([1 2 3], shape=(3,), dtype=int32)
tf.Tensor([1 2 3 4 5], shape=(5,), dtype=int32)


<br>
<br>

#### **Pass tensors instead of python literals**

<font style="color:maroon;font-size:110%">**Often, Python arguments are used to control hyperparameters and graph constructions** - for example, `num_layers=10` or `training=True` or `nonlinearity='relu'`. So, if the Python argument changes, it makes sense that you'd have to retrace the graph.

**However, it's possible that a Python argument is not being used to control graph construction**. In these cases, a change in the Python value can trigger needless retracing.</font> Take, for example, this training loop, which AutoGraph will dynamically unroll. Despite the multiple traces, the generated graph is actually identical, so retracing is unnecessary.

In [13]:
def train_one_step():
    pass

@tf.function
def train(num_steps):
    print("Tracing with num_steps = ", num_steps)
    
    tf.print("Executing with num_steps = ", num_steps)
    for _ in tf.range(num_steps):
        train_one_step()

print("Retracing occurs for different Python arguments.")
train(num_steps=10)
print()
train(num_steps=20)

print()
print("Traces are reused for Tensor arguments.")
train(num_steps=tf.constant(10))
train(num_steps=tf.constant(20))

Retracing occurs for different Python arguments.
Tracing with num_steps =  10
Executing with num_steps =  10

Tracing with num_steps =  20
Executing with num_steps =  20

Traces are reused for Tensor arguments.
Tracing with num_steps =  Tensor("num_steps:0", shape=(), dtype=int32)
Executing with num_steps =  10
Executing with num_steps =  20


<br>

<font style="color:maroon;font-size:110%">If you need to force retracing, create a new `Function`. Separate `Function` objects are guaranteed not to share traces.</font>

In [14]:
def f():
    print('Tracing!')
    tf.print('Executing')

tf.function(f)()
print()
tf.function(f)()

Tracing!
Executing

Tracing!
Executing


<br>
<br>

#### Use the tracing protocol

<font style="color:maroon;font-size:110%">Where possible, you should prefer converting the Python type into a [`tf.experimental.ExtensionType`](www.tensorflow.org/api_docs/python/tf/experimental/ExtensionType) instead. Moreover, the `TraceType` of an `ExtensionType` is the `tf.TypeSpec` associated with it. Therefore, if needed, you can simply [override](https://www.tensorflow.org/guide/extension_type#customizing_the_extensiontypes_typespec) the default `tf.TypeSpec` to take control of an `ExtensionType`'s `Tracing Protocol`.

Otherwise, for direct control over when `Function` should retrace in regards to a particular Python type, you can implement the `Tracing Protocol` for it yourself.</font>

In [15]:
@tf.function
def get_mixed_flavor(fruit_a, fruit_b):
    return fruit_a.flavor + fruit_b.flavor

class Fruit:
    flavor = tf.constant([0, 0])

class Apple(Fruit):
    flavor = tf.constant([1, 2])

class Mango(Fruit):
    flavor = tf.constant([3, 4])

# As described in the above rules, a generic TraceType for `Apple` and `Mango`
# is generated (and a corresponding ConcreteFunction is traced) but it fails to 
# match the second function call since the first pair of Apple() and Mango() 
# have gone out out of scope by then and deleted.
get_mixed_flavor(Apple(), Mango()) # Traces a new concrete function
get_mixed_flavor(Apple(), Mango()) # Traces a new concrete function again

# However, we, as the designers of the `Fruit` class, know that each subclass
# has a fixed flavor and we can reuse an existing traced concrete function if
# it was the same subclass. Avoiding such unnecessary tracing of concrete
# functions can have significant performance benefits.

class FruitTraceType(tf.types.experimental.TraceType):
# AttributeError: module 'tensorflow._api.v2.types.experimental' has no attribute 'TraceType'
# 因为 tf.types.experimental.TraceType 来自 TensorFlow Core v2.9.0

    def __init__(self, fruit_type):
        self.fruit_type = fruit_type

    def is_subtype_of(self, other):
        return (type(other) is FruitTraceType and self.fruit_type is other.fruit_type)

    def most_specific_common_supertype(self, others):
        return self if all(self == other for other in others) else None

    def __eq__(self, other):
        return type(other) is FruitTraceType and self.fruit_type == other.fruit_type
  
    def __hash__(self):
        return hash(self.fruit_type)



class FruitWithTraceType:

    def __tf_tracing_type__(self, context):
        return FruitTraceType(type(self))

class AppleWithTraceType(FruitWithTraceType):
    flavor = tf.constant([1, 2])

class MangoWithTraceType(FruitWithTraceType):
    flavor = tf.constant([3, 4])

# Now if we try calling it again:
get_mixed_flavor(AppleWithTraceType(), MangoWithTraceType()) # Traces a new concrete function
get_mixed_flavor(AppleWithTraceType(), MangoWithTraceType()) # Re-uses the traced concrete function

AttributeError: module 'tensorflow._api.v2.types.experimental' has no attribute 'TraceType'

<br>
<br>

### Obtaining concrete functions

<font style="color:maroon;font-size:120%">Every time a function is traced, a new concrete function is created. You can directly obtain a concrete function, by using `get_concrete_function`.</font>

In [16]:
print("Obtaining concrete trace")
double_strings = double.get_concrete_function(tf.constant("a"))

print("Executing traced function")
print(double_strings(tf.constant("a")))
print(double_strings(a=tf.constant("b")))

Obtaining concrete trace
Executing traced function
tf.Tensor(b'aa', shape=(), dtype=string)
tf.Tensor(b'bb', shape=(), dtype=string)


In [17]:
# You can also call get_concrete_function on an InputSpec
double_strings_from_inputspec = double.get_concrete_function(tf.TensorSpec(shape=[], 
                                                                           dtype=tf.string))
print(double_strings_from_inputspec(tf.constant("c")))

tf.Tensor(b'cc', shape=(), dtype=string)


<br>

<font style="color:maroon;font-size:110%">Printing a `ConcreteFunction` displays a summary of its input arguments (with types) and its output type.</font>

In [18]:
print(double_strings)

ConcreteFunction double(a)
  Args:
    a: string Tensor, shape=()
  Returns:
    string Tensor, shape=()


<br>

You can also directly retrieve a concrete function's signature.

In [19]:
print(double_strings.structured_input_signature)
print(double_strings.structured_outputs)

((TensorSpec(shape=(), dtype=tf.string, name='a'),), {})
Tensor("Identity:0", shape=(), dtype=string)


<br>

Using a concrete trace with incompatible types will throw an error

In [20]:
with assert_raises(tf.errors.InvalidArgumentError):
    double_strings(tf.constant(1))

Caught expected exception 
  <class 'tensorflow.python.framework.errors_impl.InvalidArgumentError'>:


Traceback (most recent call last):
  File "C:\Users\18617\AppData\Local\Temp/ipykernel_12952/1818809857.py", line 8, in assert_raises
    yield
  File "C:\Users\18617\AppData\Local\Temp/ipykernel_12952/411856298.py", line 2, in <module>
    double_strings(tf.constant(1))
tensorflow.python.framework.errors_impl.InvalidArgumentError: cannot compute __inference_double_162 as input #0(zero-based) was expected to be a string tensor but is a int32 tensor [Op:__inference_double_162]


<br>

<font style="color:maroon;font-size:110%">You may notice that Python arguments are given special treatment in a concrete function's input signature.</font> Prior to TensorFlow 2.3, Python arguments were simply removed from the concrete function's signature. <font style="color:maroon;font-size:110%">Starting with TensorFlow 2.3, Python arguments remain in the signature, but are constrained to take the value set during tracing.</font>

In [21]:
@tf.function
def pow(a, b):
    return a ** b

square = pow.get_concrete_function(a=tf.TensorSpec(None, tf.float32), b=2)
print(square)

ConcreteFunction pow(a, b=2)
  Args:
    a: float32 Tensor, shape=<unknown>
  Returns:
    float32 Tensor, shape=<unknown>


In [22]:
assert square(tf.constant(10.0)) == 100

with assert_raises(TypeError):
    square(tf.constant(10.0), b=3)

Caught expected exception 
  <class 'TypeError'>:


Traceback (most recent call last):
  File "D:\Programs\Anaconda3\envs\tfg\lib\site-packages\tensorflow\python\eager\function.py", line 1614, in _call_impl
    return self._call_with_flat_signature(args, kwargs,
  File "D:\Programs\Anaconda3\envs\tfg\lib\site-packages\tensorflow\python\eager\function.py", line 1659, in _call_with_flat_signature
    raise TypeError(f"{self._flat_signature_summary()} got unexpected "
TypeError: pow(a) got unexpected keyword arguments: b.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\18617\AppData\Local\Temp/ipykernel_12952/1818809857.py", line 8, in assert_raises
    yield
  File "C:\Users\18617\AppData\Local\Temp/ipykernel_12952/1293241696.py", line 4, in <module>
    square(tf.constant(10.0), b=3)
TypeError: ConcreteFunction pow(a, b) was constructed with int value 2 in b, but was called with int value 3.


<br>
<br>

### Obtaining graphs

Each concrete function is a callable wrapper around a `tf.Graph`. Although retrieving the actual `tf.Graph` object is not something you'll normally need to do, you can obtain it easily from any concrete function.

In [23]:
graph = double_strings.graph
for node in graph.as_graph_def().node:
    print(f'{node.input} -> {node.name}')


[] -> a
['a', 'a'] -> add
['add'] -> Identity


<br>
<br>

### **Debugging**

<font style="color:maroon;font-size:110%">**In general, debugging code is easier in eager mode than inside `tf.function`**. You should ensure that your code executes error-free in eager mode before decorating with `tf.function`. To assist in the debugging process, you can call `tf.config.run_functions_eagerly(True)` to globally disable and reenable `tf.function`.</font>

<br>

<font style="color:maroon;font-size:115%">When tracking down issues that only appear within `tf.function`, here are some tips:</font>
- Plain old Python `print` calls only execute during tracing, helping you track down when your function gets (re)traced.
- `tf.print` calls will execute every time, and can help you track down intermediate values during execution.
- `tf.debugging.enable_check_numerics` is an easy way to track down where NaNs and Inf are created.
- `pdb` (the [Python debugger](https://docs.python.org/3/library/pdb.html)) can help you understand what's going on during tracing. (Caveat: `pdb` will drop you into AutoGraph-transformed source code.)

<br>
<br>
<br>
<br>

## **AutoGraph transformations**

<font style="color:maroon;font-size:110%">AutoGraph is a library that is on by default in `tf.function`, and transforms a subset of Python eager code into graph-compatible TensorFlow ops. This includes control flow like `if`, `for`, `while`.

TensorFlow ops like `tf.cond` and `tf.while_loop` continue to work, but control flow is often easier to write and understand when written in Python.</font>

In [24]:
# A simple loop

@tf.function
def f(x):
    while tf.reduce_sum(x) > 1:
        tf.print(x)
        x = tf.tanh(x)
    return x

f(tf.random.uniform([5]))

[0.632292867 0.942353725 0.0461101532 0.224100947 0.324224234]
[0.559629 0.736301839 0.0460774973 0.220423281 0.313321471]
[0.507702112 0.62690562 0.0460449122 0.216921449 0.30345574]
[0.468152851 0.555917859 0.0460123979 0.213581875 0.29447186]
[0.436705619 0.504942358 0.0459799431 0.210392356 0.286245465]
[0.410909981 0.465995133 0.0459475629 0.207342014 0.278675467]
[0.389245063 0.434957772 0.0459152497 0.204420939 0.271678746]
[0.370709181 0.409456253 0.0458830073 0.201620296 0.265186191]
[0.354611874 0.388010919 0.0458508283 0.198931992 0.259139895]
[0.340459198 0.369644165 0.04581872 0.196348682 0.253490835]
[0.327887267 0.353680432 0.0457866788 0.193863705 0.248197258]
[0.316621065 0.339635491 0.0457547 0.191470981 0.243223295]
[0.30644846 0.327151924 0.0457227938 0.189164937 0.238537937]
[0.297202677 0.315959334 0.0456909537 0.186940446 0.234114304]
[0.288750589 0.305848777 0.0456591807 0.184792802 0.229928792]
[0.280984461 0.296655864 0.0456274711 0.182717681 0.225960776]
[0.2

<tf.Tensor: shape=(5,), dtype=float32, numpy=
array([0.267172  , 0.2805226 , 0.04556425, 0.1787693 , 0.21860625],
      dtype=float32)>

<br>

If you're curious you can inspect the code autograph generates.

In [25]:
print(tf.autograph.to_code(f.python_function))

def tf__f(x):
    with ag__.FunctionScope('f', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()

        def get_state():
            return (x,)

        def set_state(vars_):
            nonlocal x
            (x,) = vars_

        def loop_body():
            nonlocal x
            ag__.converted_call(ag__.ld(tf).print, (ag__.ld(x),), None, fscope)
            x = ag__.converted_call(ag__.ld(tf).tanh, (ag__.ld(x),), None, fscope)

        def loop_test():
            return ag__.converted_call(ag__.ld(tf).reduce_sum, (ag__.ld(x),), None, fscope) > 1
        ag__.while_stmt(loop_test, loop_body, get_state, set_state, ('x',), {})
        try:
            do_return = True
            retval_ = ag__.ld(x)
        except:
            do_return = False
            raise
        return fscope.ret(retval_, do_return)



<br>
<br>

### Conditionals

AutoGraph will convert some `if <condition>` statements into the equivalent `tf.cond` calls. This substitution is made if `<condition>` is a Tensor. Otherwise, the `if` statement is executed as a Python conditional.

A **Python conditional** executes during tracing, so **exactly one branch of the conditional** will be added to the graph. <font style="color:maroon;font-size:110%">Without AutoGraph, this traced graph would be unable to take the alternate branch if there is data-dependent control flow.</font>

`tf.cond` traces and adds both branches of the conditional to the graph, dynamically selecting a branch at execution time. Tracing can have unintended side effects; check out [AutoGraph tracing effects](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/autograph/g3doc/reference/control_flow.md#effects-of-the-tracing-process) for more information.

In [26]:
@tf.function
def fizzbuzz(n):
    for i in tf.range(1, n + 1):
        print('Tracing for loop')
        if i % 15 == 0:
            print('Tracing fizzbuzz branch')
            tf.print('fizzbuzz')
        elif i % 3 == 0:
            print('Tracing fizz branch')
            tf.print('fizz')
        elif i % 5 == 0:
            print('Tracing buzz branch')
            tf.print('buzz')
        else:
            print('Tracing default branch')
            tf.print(i)

fizzbuzz(tf.constant(5))
print('\n\n')
fizzbuzz(tf.constant(20))

Tracing for loop
Tracing fizzbuzz branch
Tracing fizz branch
Tracing buzz branch
Tracing default branch
1
2
fizz
4
buzz



1
2
fizz
4
buzz
fizz
7
8
fizz
buzz
11
fizz
13
14
fizzbuzz
16
17
fizz
19
buzz


<br>

See the [reference documentation](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/autograph/g3doc/reference/control_flow.md#if-statements) for additional restrictions on AutoGraph-converted if statements.

<br>
<br>

### Loops

AutoGraph will convert some `for` and `while` statements into the equivalent **TensorFlow looping ops**, like `tf.while_loop`. If not converted, the `for` or `while` loop is executed as a **Python loop**.

This substitution is made in the following situations:

- `for x in y`: if `y` is a Tensor, convert to `tf.while_loop`. In the special case where `y` is a `tf.data.Dataset`, a combination of `tf.data.Dataset` ops are generated.
- `while <condition>`: if `<condition>` is a Tensor, convert to `tf.while_loop`.

<font style="color:maroon;font-size:110%">A **Python loop** executes during tracing, adding additional ops to the `tf.Graph` for every iteration of the loop.

A **TensorFlow loop** traces the body of the loop, and dynamically selects how many iterations to run at execution time.  The loop body only appears once in the generated `tf.Graph`.</font>

See the [reference documentation](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/autograph/g3doc/reference/control_flow.md#while-statements) for additional restrictions on AutoGraph-converted `for` and `while` statements.

<br>

#### Looping over Python data

<font style="color:maroon;font-size:110%">A common pitfall is to loop over Python/NumPy data within a `tf.function`. This loop will execute during the tracing process, adding a copy of your model to the `tf.Graph` for each iteration of the loop.

If you want to wrap the entire training loop in `tf.function`, the safest way to do this is to wrap your data as a `tf.data.Dataset` so that AutoGraph will dynamically unroll the training loop.</font>

In [27]:
def measure_graph_size(f, *args):
    g = f.get_concrete_function(*args).graph
    print("{}({}) contains {} nodes in its graph".format(f.__name__, 
                                                         ', '.join(map(str, args)), 
                                                         len(g.as_graph_def().node)))

@tf.function
def train(dataset):
    loss = tf.constant(0)
    for x, y in dataset:
        loss += tf.abs(y - x) # Some dummy computation.
    return loss

small_data = [(1, 1)] * 3
big_data = [(1, 1)] * 10
measure_graph_size(train, small_data)
print()
measure_graph_size(train, big_data)
print()

measure_graph_size(train, tf.data.Dataset.from_generator(
    lambda: small_data, (tf.int32, tf.int32)))
print()

measure_graph_size(train, tf.data.Dataset.from_generator(
    lambda: big_data, (tf.int32, tf.int32)))

train([(1, 1), (1, 1), (1, 1)]) contains 11 nodes in its graph

train([(1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1)]) contains 32 nodes in its graph

train(<FlatMapDataset element_spec=(TensorSpec(shape=<unknown>, dtype=tf.int32, name=None), TensorSpec(shape=<unknown>, dtype=tf.int32, name=None))>) contains 6 nodes in its graph

train(<FlatMapDataset element_spec=(TensorSpec(shape=<unknown>, dtype=tf.int32, name=None), TensorSpec(shape=<unknown>, dtype=tf.int32, name=None))>) contains 6 nodes in its graph


<br>

<font style="color:maroon;font-size:110%">When wrapping Python/NumPy data in a Dataset, be mindful of `tf.data.Dataset.from_generator` versus ` tf.data.Dataset.from_tensors`. 
* The former will keep the data in Python and fetch it via `tf.py_function` which can have **performance implications** `(性能可能会比较差)`, 
* whereas the latter will bundle a copy of the data as one large `tf.constant()` node in the graph, which can have **memory implications** `(内存占用可能会比较大)`.
<br>
<br>

Reading data from files via `TFRecordDataset`, `CsvDataset`, etc. is the most effective way to consume data, as then TensorFlow itself can manage the asynchronous loading and prefetching of data, without having to involve Python. To learn more, see the [`tf.data`: Build TensorFlow input pipelines](https://github.com/tensorflow/docs/blob/master/site/guide/data) guide.</font>

<br>
<br>

#### Accumulating values in a loop

<font style="color:maroon;font-size:110%">A common pattern is to accumulate intermediate values from a loop. Normally, this is accomplished by appending to a Python list or adding entries to a Python dictionary. **However, as these are Python side effects, they will not work as expected in a dynamically unrolled loop**. **Use `tf.TensorArray` to accumulate results from a dynamically unrolled loop**.</font>

In [28]:
batch_size = 2
seq_len = 3
feature_size = 4

def rnn_step(inp, state):
    return inp + state



@tf.function
def dynamic_rnn(rnn_step, input_data, initial_state):
    # [batch, time, features] -> [time, batch, features]
    input_data = tf.transpose(input_data, [1, 0, 2])
    max_seq_len = input_data.shape[0]

    states = tf.TensorArray(tf.float32, size=max_seq_len)
    state = initial_state
    for i in tf.range(max_seq_len):
        state = rnn_step(input_data[i], state)
        states = states.write(i, state)
    return tf.transpose(states.stack(), [1, 0, 2])

dynamic_rnn(rnn_step,
            tf.random.uniform([batch_size, seq_len, feature_size]),
            tf.zeros([batch_size, feature_size]))

<tf.Tensor: shape=(2, 3, 4), dtype=float32, numpy=
array([[[0.43029308, 0.5712124 , 0.60114074, 0.75877225],
        [1.3319141 , 1.4822532 , 1.2005941 , 1.0545665 ],
        [1.957657  , 2.3078313 , 1.9735879 , 1.5388166 ]],

       [[0.47058594, 0.5875126 , 0.25433707, 0.20146513],
        [1.1806917 , 1.2655056 , 0.47434974, 0.67644346],
        [1.1937488 , 2.1285586 , 1.2009484 , 1.2030194 ]]], dtype=float32)>

<br>
<br>
<br>

## <font color=maroon>**Limitations**</font>

TensorFlow `Function` has a few limitations by design that you should be aware of when converting a Python function to a `Function`.

<br>

### **Executing Python side effects**

<font style="color:maroon;font-size:110%"><ins>Side effects</ins>, like **printing**, **appending to lists**, and **mutating globals**, can behave unexpectedly inside a `Function`, sometimes executing twice or not all.</font> They only happen the first time you call a `Function` with a set of inputs.  Afterwards, the traced `tf.Graph` is reexecuted, without executing the Python code.

The general rule of thumb is to avoid relying on Python side effects in your logic and only use them to debug your traces. Otherwise, TensorFlow APIs like `tf.data`, `tf.print`, `tf.summary`, `tf.Variable.assign`, and `tf.TensorArray` are the best way to ensure your code will be executed by the TensorFlow runtime with each call.

In [29]:
@tf.function
def f(x):
    print("Traced with", x)
    tf.print("Executed with", x)

f(1)
print()

f(1)
print()

f(2)

Traced with 1
Executed with 1

Executed with 1

Traced with 2
Executed with 2


<font style="color:maroon;font-size:110%">If you would like to execute Python code during each invocation of a `Function`, **`tf.py_function` is an exit hatch**. The **drawback** of `tf.py_function` is that it's not portable or particularly performant, cannot be saved with SavedModel, and does not work well in distributed (multi-GPU, TPU) setups. Also, since `tf.py_function` has to be wired into the graph, it casts all inputs/outputs to tensors.</font>

<br>
<br>

#### Changing Python global and free variables

<font style="color:maroon;font-size:110%">Changing Python global and [free variables](https://docs.python.org/3/reference/executionmodel.html#binding-of-names) counts as a Python side effect, so it only happens during tracing.</font>


In [30]:
external_list = []

@tf.function
def side_effect(x):
    print('Python side effect')
    external_list.append(x)

side_effect(1)
print()

side_effect(1)
print()

side_effect(1)

# The list append only happened once!
assert len(external_list) == 1

Python side effect




In [31]:
assert len(external_list) == 2

AssertionError: 

<br>

Sometimes unexpected behaviors are very hard to notice. In the example below, the `counter` is intended to safeguard the increment of a variable. However because it is a python integer and not a TensorFlow object, it's value is captured during the first trace. When the `tf.function` is used, the `assign_add` will be recorded unconditionally in the underlying graph. Therefore `v` will increase by 1, every time the `tf.function` is called. 

<font style="color:maroon;font-size:110%">This issue is common among users that try to migrate their Grpah-mode Tensorflow code to Tensorflow 2 using `tf.function` decorators, when python side-effects (the `counter` in the example) are used to determine what ops to run (`assign_add` in the example).</font> Usually, users realize this only after seeing suspicious numerical results, or significantly lower performance than expected (e.g. if the guarded operation is very costly).

In [32]:
class Model(tf.Module):
    def __init__(self):
        self.v = tf.Variable(0)
        self.counter = 0

    @tf.function
    def __call__(self):
        if self.counter == 0:
            # A python side-effect
            self.counter += 1
            self.v.assign_add(1)

        return self.v

m = Model()
for n in range(3):
    print(m().numpy()) # prints 1, 2, 3

1
2
3


In [33]:
class Model(tf.Module):
    def __init__(self):
        self.v = tf.Variable(0)
        self.counter = 0

    @tf.function
    def __call__(self):
        if self.counter == 0:
            # A python side-effect
            self.counter += 1
            self.v.assign_add(1)

        return self.counter, self.v

m = Model()
for n in range(3):
    c,v = m()
    print(c.numpy(), v.numpy())

1 1
1 2
1 3


<br>

<font style="color:maroon;font-size:110%">A workaround to achieve the expected behavior is **using [`tf.init_scope`](https://www.tensorflow.org/api_docs/python/tf/init_scope) to lift the operations outside of the function graph**. This ensures that the variable increment is only done once during tracing time. It should be noted `init_scope` has other side effects including cleared control flow and gradient tape. Sometimes the usage of `init_scope` can become too complex to manage realistically.</font>

In [34]:
class Model(tf.Module):
    def __init__(self):
        self.v = tf.Variable(0)
        self.counter = 0

    @tf.function
    def __call__(self):
        if self.counter == 0:
            # Lifts ops out of function-building graphs
            with tf.init_scope():
                self.counter += 1
                self.v.assign_add(1)

        return self.v

m = Model()
for n in range(3):
    print(m().numpy()) # prints 1, 1, 1

1
1
1


<br>

<font style="color:maroon;font-size:120%"><ins>In summary</ins>, as a rule of thumb, you should avoid mutating python objects such as integers or containers like lists that live outside the `Function`. **Instead, use arguments and TF objects**. For example, the section ["Accumulating values in a loop"](#accumulating_values_in_a_loop) has one example of how list-like operations can be implemented.

You can, in some cases, capture and manipulate state if it is a [`tf.Variable`](https://www.tensorflow.org/guide/variable). This is how the weights of Keras models are updated with repeated calls to the same `ConcreteFunction`.</font>

<br>
<br>

#### Using Python iterators and generators

Many Python features, such as generators and iterators, rely on the Python runtime to keep track of state. In general, while these constructs work as expected in eager mode, they are examples of Python side effects and therefore only happen during tracing.

In [35]:
@tf.function
def buggy_consume_next(iterator):
    tf.print("Value:", next(iterator))

iterator = iter([1, 2, 3])
buggy_consume_next(iterator)
print()

# This reuses the first value from the iterator, rather than consuming the next value.
buggy_consume_next(iterator)
print()

buggy_consume_next(iterator)


Value: 1

Value: 1

Value: 1


<br>

<font style="color:maroon;font-size:120%">Just like how TensorFlow has a specialized `tf.TensorArray` for list constructs, it has a specialized `tf.data.Iterator` for iteration constructs.<br>
See the section on [AutoGraph transformations](#autograph_transformations) for an overview. Also, the [`tf.data`](https://www.tensorflow.org/guide/data) API can help implement generator patterns:</font>

In [36]:
@tf.function
def good_consume_next(iterator):
  # This is ok, iterator is a tf.data.Iterator
  tf.print("Value:", next(iterator))

ds = tf.data.Dataset.from_tensor_slices([1, 2, 3])
iterator = iter(ds)

good_consume_next(iterator)
good_consume_next(iterator)
good_consume_next(iterator)

Value: 1
Value: 2
Value: 3


<br>
<br>

### All outputs of a tf.function must be return values

<font style="color:maroon;font-size:120%">With the exception of `tf.Variable`s, a tf.function must return all its outputs. **Attempting to directly access any tensors from a function without going through return values causes "leaks"**.</font>

<font style="font-size:120%">For example, the function below "leaks" the tensor `a` through the Python global `x`:</font>

In [37]:
x = None
print('type(x): ', type(x), '\n')        # 陈锐平添加

@tf.function
def leaky_function(a):
    global x
    x = a + 1             # Bad - leaks local tensor
                          # 因为下面的 x.numpy() 是在使用 x，但是 x 并不是 tf.function leaky_function() 的返回值。
    return a + 2

correct_a = leaky_function(tf.constant(1))

print(correct_a.numpy())  # Good - value obtained from function's returns
print()

try:
    print('type(x): ', type(x), '\n')    # 陈锐平添加
    x.numpy()             # Bad - tensor leaked from inside the function, cannot be used here
except AttributeError as expected:
    print(expected)

type(x):  <class 'NoneType'> 

3

type(x):  <class 'tensorflow.python.framework.ops.Tensor'> 

'Tensor' object has no attribute 'numpy'


<br>

<font style="color:maroon;font-size:150%">This is true even if the leaked value is also returned:</font>

In [38]:
@tf.function
def leaky_function(a):
    global x
    x = a + 1  # Bad - leaks local tensor，
               # 因为下面的 x.numpy() 是直接使用 x，而不是通过 tf.function leaky_function() 的返回值的形式。
    return x   # Good - uses local tensor

correct_a = leaky_function(tf.constant(1))

print(correct_a.numpy())  # Good - value obtained from function's returns
print()

try:
    x.numpy()             # Bad - tensor leaked from inside the function, cannot be used here
except AttributeError as expected:
    print(expected)

print()

@tf.function
def captures_leaked_tensor(b):
    b += x               # Bad - `x` is leaked from `leaky_function`
                         # 因为这里直接引用了 x，而不是通过 tf.function leaky_function() 的返回值的形式。
    return b

with assert_raises(TypeError):
    captures_leaked_tensor(tf.constant(2))

2

'Tensor' object has no attribute 'numpy'

Caught expected exception 
  <class 'TypeError'>:


Traceback (most recent call last):
  File "C:\Users\18617\AppData\Local\Temp/ipykernel_12952/1818809857.py", line 8, in assert_raises
    yield
  File "C:\Users\18617\AppData\Local\Temp/ipykernel_12952/3689345937.py", line 27, in <module>
    captures_leaked_tensor(tf.constant(2))
TypeError: <tf.Tensor 'add:0' shape=() dtype=int32> is out of scope and cannot be used here. Use return values, explicit Python locals or TensorFlow collections to access it.
Please see https://www.tensorflow.org/guide/function#all_outputs_of_a_tffunction_must_be_return_values for more information.

<tf.Tensor 'add:0' shape=() dtype=int32> was defined here:
    File "D:\Programs\Anaconda3\envs\tfg\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "D:\Programs\Anaconda3\envs\tfg\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "D:\Programs\Anaconda3\envs\tfg\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      ap

<br>

<font style="color:maroon;font-size:120%">Usually, leaks such as these occur when you use Python statements or data structures.
In addition to leaking inaccessible tensors, such statements are also likely wrong because they count as Python side effects, and are not guaranteed to execute at every function call.

Common ways to leak local tensors also include mutating an external Python collection, or an object:</font>

In [39]:
class MyClass:

    def __init__(self):
        self.field = None

external_list = []

external_object = MyClass()


def leaky_function():
    a = tf.constant(1)
    external_list.append(a)    # Bad - leaks tensor
    external_object.field = a  # Bad - leaks tensor

<br>
<br>

### Recursive tf.functions are not supported

Recursive `Function`s are not supported and could cause infinite loops. For example,

In [ ]:
@tf.function
def recursive_fn(n):
    if n > 0:
        return recursive_fn(n - 1)
    else:
        return 1

with assert_raises(Exception):
    recursive_fn(tf.constant(5))  # Bad - maximum recursion error.

# 运行这个 cell 的结果：
# The kernel appears to have died. It will restart automatically.

<br>

<font style="color:maroon;font-size:120%">Even if a recursive `Function` seems to work, the python function will be traced multiple times and could have performance implication.</font> For example,

In [40]:
@tf.function
def recursive_fn(n):
    if n > 0:
        print('tracing')
        return recursive_fn(n - 1)
    else:
        return 1

recursive_fn(5)  # Warning - multiple tracings

tracing
tracing
tracing
tracing
tracing


<tf.Tensor: shape=(), dtype=int32, numpy=1>

<br>
<br>
<br>

## <font color=maroon>**Known Issues**</font>

If your `Function` is not evaluating correctly, the error may be explained by these known issues which are planned to be fixed in the future.

<br>

### Depending on Python global and free variables

<font style="color:maroon;font-size:120%">**`Function` creates a new `ConcreteFunction` when called with a new value of a Python argument. However, it does not do that for the Python closure, globals, or nonlocals of that `Function`**. If their value changes in between calls to the `Function`, the `Function` will still use the values they had when it was traced. This is different from how regular Python functions work.

For that reason, you should follow a functional programming style that uses arguments instead of closing over outer names.</font>

In [41]:
@tf.function
def buggy_add():             # foo 变量不是 buggy_add() 函数的形参（不是局部变量）
    return 1 + foo

@tf.function
def recommended_add(foo):    # foo 变量是 recommended_add() 函数的形参
    return 1 + foo

foo = 1
print("Buggy:", buggy_add())
print()

print("Correct:", recommended_add(foo))

Buggy: tf.Tensor(2, shape=(), dtype=int32)

Correct: tf.Tensor(2, shape=(), dtype=int32)


In [42]:
print("Updating the value of `foo` to 100!")
foo = 100
print("Buggy:", buggy_add())  # Did not change!
print()

print("Correct:", recommended_add(foo))

Updating the value of `foo` to 100!
Buggy: tf.Tensor(2, shape=(), dtype=int32)

Correct: tf.Tensor(101, shape=(), dtype=int32)


<br>

<font style="color:maroon;font-size:120%">Another way to update a global value, is to make it a `tf.Variable` and use the `Variable.assign` method instead.</font>

In [43]:
@tf.function
def variable_add():
    return 1 + foo

foo = tf.Variable(1)
print("Variable:", variable_add())

Variable: tf.Tensor(2, shape=(), dtype=int32)


In [44]:
print("Updating the value of `foo` to 100!")
foo.assign(100)
print("Variable:", variable_add())

Updating the value of `foo` to 100!
Variable: tf.Tensor(101, shape=(), dtype=int32)


<br>

#### Depending on Python objects

<font style="color:maroon;font-size:120%">The recommendation to pass Python objects as arguments into `tf.function` has a number of known issues, that are expected to be fixed in the future. 

In general, you can rely on consistent tracing if you use a Python primitive or `tf.nest`-compatible structure as an argument or pass in a *different* instance of an object into a `Function`. However, `Function` will *not* create a new trace when you pass **the same object and only change its attributes**.</font>

In [45]:
class SimpleModel(tf.Module):
    def __init__(self):
        # These values are *not* tf.Variables.
        self.bias = 0.
        self.weight = 2.

@tf.function
def evaluate(model, x):
    return model.weight * x + model.bias


simple_model = SimpleModel()
x = tf.constant(10.)
print(evaluate(simple_model, x))

tf.Tensor(20.0, shape=(), dtype=float32)


In [46]:
print("Adding bias!")
simple_model.bias += 5.0
print(evaluate(simple_model, x))  # Didn't change :(

Adding bias!
tf.Tensor(20.0, shape=(), dtype=float32)


<br>

<font style="color:maroon;font-size:120%">Using the same `Function` to evaluate the updated instance of the model will be buggy since the updated model has the [same cache key](#rules_of_tracing) as the original model.

For that reason, **you're recommended to write your `Function` to avoid depending on mutable object attributes or create new objects**.

**If that is not possible, one workaround is to make new `Function`s each time you modify your object to force retracing**:</font>

In [47]:
def evaluate(model, x):
    return model.weight * x + model.bias

new_model = SimpleModel()
evaluate_no_bias = tf.function(evaluate).get_concrete_function(new_model, x)

# Don't pass in `new_model`, `Function` already captured its state during tracing.
print(evaluate_no_bias(x))  

tf.Tensor(20.0, shape=(), dtype=float32)


In [48]:
print("Adding bias!")
new_model.bias += 5.0

# Create new Function and ConcreteFunction since you modified new_model.
evaluate_with_bias = tf.function(evaluate).get_concrete_function(new_model, x)
print(evaluate_with_bias(x)) # Don't pass in `new_model`.

Adding bias!
tf.Tensor(25.0, shape=(), dtype=float32)


<br>

<font style="color:maroon;font-size:120%">As [retracing can be expensive](https://www.tensorflow.org/guide/intro_to_graphs#tracing_and_performance), you can use `tf.Variable`s as object attributes, which can be mutated (but not changed, careful!) for a similar effect without needing a retrace.</font>

In [49]:
class BetterModel:

    def __init__(self):
        self.bias = tf.Variable(0.)
        self.weight = tf.Variable(2.)

@tf.function
def evaluate(model, x):
    return model.weight * x + model.bias

better_model = BetterModel()
print(evaluate(better_model, x))

tf.Tensor(20.0, shape=(), dtype=float32)


In [50]:
print("Adding bias!")
better_model.bias.assign_add(5.0)  # Note: instead of better_model.bias += 5
print(evaluate(better_model, x))   # This works!

Adding bias!
tf.Tensor(25.0, shape=(), dtype=float32)


<br>
<br>

### Creating tf.Variables

<font style="color:maroon;font-size:120%">`Function` only supports singleton `tf.Variable`s created once on the first call, and reused across subsequent function calls. The code snippet below would create a new `tf.Variable` in every function call, which results in a `ValueError` exception.</font>

Example:

In [51]:
@tf.function
def f(x):
    v = tf.Variable(1.0)
    return v

with assert_raises(ValueError):
    f(1.0)

Caught expected exception 
  <class 'ValueError'>:


Traceback (most recent call last):
  File "C:\Users\18617\AppData\Local\Temp/ipykernel_12952/1818809857.py", line 8, in assert_raises
    yield
  File "C:\Users\18617\AppData\Local\Temp/ipykernel_12952/847243905.py", line 7, in <module>
    f(1.0)
ValueError: in user code:

    File "C:\Users\18617\AppData\Local\Temp/ipykernel_12952/847243905.py", line 3, in f  *
        v = tf.Variable(1.0)

    ValueError: tf.function only supports singleton tf.Variables created on the first call. Make sure the tf.Variable is only created once or created outside tf.function. See https://www.tensorflow.org/guide/function#creating_tfvariables for more information.



<br>

<font style="color:maroon;font-size:120%">A common pattern used to work around this limitation is to start with a Python None value, then conditionally create the `tf.Variable` if the value is None:</font>

In [52]:
class Count(tf.Module):
    def __init__(self):
        self.count = None

    @tf.function
    def __call__(self):
        if self.count is None:
            self.count = tf.Variable(0)
        return self.count.assign_add(1)

c = Count()
print(c())

print()
print(c())

tf.Tensor(1, shape=(), dtype=int32)

tf.Tensor(2, shape=(), dtype=int32)


<br>
<br>

#### Using with multiple Keras optimizers
<font style="color:maroon;font-size:120%">You may encounter `ValueError: tf.function only supports singleton tf.Variables created on the first call.` when using more than one Keras optimizer with a `tf.function`. 

This error occurs because **optimizers internally create `tf.Variables` when they apply gradients for the first time**.</font>

In [53]:
opt1 = tf.keras.optimizers.Adam(learning_rate = 1e-2)
opt2 = tf.keras.optimizers.Adam(learning_rate = 1e-3)
 
@tf.function
def train_step(w, x, y, optimizer):
    with tf.GradientTape() as tape:
        L = tf.reduce_sum(tf.square(w*x - y))
    gradients = tape.gradient(L, [w])
    optimizer.apply_gradients(zip(gradients, [w]))

w = tf.Variable(2.)
x = tf.constant([-1.])
y = tf.constant([2.])

train_step(w, x, y, opt1)
print("Calling `train_step` with different optimizer...")
print()

with assert_raises(ValueError):
    train_step(w, x, y, opt2)

Calling `train_step` with different optimizer...

Caught expected exception 
  <class 'ValueError'>:


Traceback (most recent call last):
  File "C:\Users\18617\AppData\Local\Temp/ipykernel_12952/1818809857.py", line 8, in assert_raises
    yield
  File "C:\Users\18617\AppData\Local\Temp/ipykernel_12952/2313663715.py", line 20, in <module>
    train_step(w, x, y, opt2)
ValueError: in user code:

    File "C:\Users\18617\AppData\Local\Temp/ipykernel_12952/2313663715.py", line 9, in train_step  *
        optimizer.apply_gradients(zip(gradients, [w]))
    File "D:\Programs\Anaconda3\envs\tfg\lib\site-packages\keras\optimizer_v2\optimizer_v2.py", line 639, in apply_gradients  **
        self._create_all_weights(var_list)
    File "D:\Programs\Anaconda3\envs\tfg\lib\site-packages\keras\optimizer_v2\optimizer_v2.py", line 823, in _create_all_weights
        _ = self.iterations
    File "D:\Programs\Anaconda3\envs\tfg\lib\site-packages\keras\optimizer_v2\optimizer_v2.py", line 830, in __getattribute__
        return super(OptimizerV2, self).__getattribute__(name)
    File "D:\Programs\Anaconda

<br>

<font style="color:maroon;font-size:120%">If you need to change the optimizer during training, a workaround is to create a new `Function` for each optimizer, calling the [`ConcreteFunction`](#obtaining_concrete_functions) directly.</font>

In [54]:
opt1 = tf.keras.optimizers.Adam(learning_rate = 1e-2)
opt2 = tf.keras.optimizers.Adam(learning_rate = 1e-3)

# Not a tf.function.
def train_step(w, x, y, optimizer):
    with tf.GradientTape() as tape:
        L = tf.reduce_sum(tf.square(w*x - y))
    gradients = tape.gradient(L, [w])
    optimizer.apply_gradients(zip(gradients, [w]))

w = tf.Variable(2.)
x = tf.constant([-1.])
y = tf.constant([2.])

# Make a new Function and ConcreteFunction for each optimizer.
train_step_1 = tf.function(train_step).get_concrete_function(w, x, y, opt1)
train_step_2 = tf.function(train_step).get_concrete_function(w, x, y, opt2)
for i in range(10):
    if i % 2 == 0:
        train_step_1(w, x, y) # `opt1` is not used as a parameter. 
    else:
        train_step_2(w, x, y) # `opt2` is not used as a parameter.

<br>
<br>

#### Using with multiple Keras models

<font style="color:maroon;font-size:120%">You may also encounter `ValueError: tf.function only supports singleton tf.Variables created on the first call.` when passing different model instances to the same `Function`.

This error occurs because **Keras models** (which [do not have their input shape defined](https://www.tensorflow.org/guide/keras/custom_layers_and_models#best_practice_deferring_weight_creation_until_the_shape_of_the_inputs_is_known)) **and Keras layers create `tf.Variables`s when they are first called**. You may be attempting to initialize those variables inside a `Function`, which has already been called. **To avoid this error, try calling `model.build(input_shape)` to initialize all the weights before training the model**.</font>

<br>
<br>
<br>

## Further reading

To learn about how to export and load a `Function`, see the [SavedModel guide](https://github.com/tensorflow/docs/blob/master/site/guide/saved_model). 

To learn more about graph optimizations that are performed after tracing, see the [Grappler guide](https://github.com/tensorflow/docs/blob/master/site/guide/graph_optimization). 

To learn how to optimize your data pipeline and profile your model, see the [Profiler guide](https://github.com/tensorflow/docs/blob/master/site/guide/profiler.md).

<br>
<br>
<br>

**Copyright 2020 The TensorFlow Authors.**

```python
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
```

<br>
<br>
<br>